In [4]:
import numpy as np
import pandas as pd
import re
import time
from datasketch import MinHash, MinHashLSHForest, MinHashLSH

In [71]:
lines = []
'''with open("../data/ftp.mokk.bme.hu/Hunglish2/modern.lit/bi/hunglish1.lit.bi", 'r', encoding='utf_8') as file:
    for l in file:
        lines.append(re.split(r'\t+', l.strip()))

db = pd.DataFrame(lines, columns=["hun", "en"])'''

with open("../data/ftp.mokk.bme.hu/Hunglish2/combined/hunglish2small-test.hu", 'r', encoding='utf_8') as file:
    for l in file:
        lines.append(l)
db = pd.DataFrame(lines, columns=["hun"])

permutations = 128
num_recommendations = 2

In [18]:
db

,hun
0,"Mindenkiben magamat látom, semmi többet és egy..."
1,- Te nem hiszel bennem - jegyezte meg.\n
2,- Hát ez mi légyen? - kérdezte Joe.\n
3,"Míg a cseresznyét szemelgette, az öregasszony ..."
4,"Míg a cseresznyét szemelgette, az öregasszony ..."
...,...
23261,"Tudod hatástalanítani, igaz?\n"
23262,Köszönöm.\n
23263,Igen.\n
23264,Az FBI nyomozott?\n


In [10]:
def preprocess(text):
    text = re.sub(r'[^\w\s]','',text)
    tokens = text.lower()
    tokens = tokens.split()
    return tokens

In [82]:
def get_forest(data, perms):
    minhash = []
    
    for text in data["hun"]:
        tokens = preprocess(text)
        m = MinHash(num_perm=perms)
        for s in tokens:
            m.update(s.encode('utf8'))
        minhash.append(m)
        
    lsh = MinHashLSH(threshold=0.7, num_perm=perms)
    for i,m in enumerate(minhash):
        lsh.insert(i,m)
    
    return lsh

In [78]:
def predict(text, database, perms, num_results, forest):
    start_time = time.time()
    
    tokens = preprocess(text)
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    idx_array = np.array(forest.query(m))

    if len(idx_array) == 0:
        return None
    
    return idx_array

In [79]:
def make_sets(l):
    ret = []
    for i in range(len(l)):
        for j in range(i+1, len(l)):
            ret.append((l[i],l[j]))
    return ret

In [83]:
forest = get_forest(db, permutations)

In [89]:
skip_indices = []
for idx in range(len(db)):
    if idx in skip_indices:
        continue
    pred = predict(db["hun"][idx], db, permutations, num_recommendations, forest)
    if len(pred) > 1:
        '''print("Original:")
        print(idx)
        print(db["hun"][idx])
        print("Preds:")
        for p in pred:
            if p != idx:
                print(p)
                print(db["hun"][p])
        print("----")'''
        for p in pred:
            if p != idx:
                skip_indices.append(p)
    #sets = make_sets(sorted(pred))
    #for s in sets:
    #    pairs.add(s)
#print(pairs)

In [91]:
len(skip_indices)

13206